# Step 1 - Scraping

In [2]:
#import dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import pandas as pd

### NASA Mars News

In [3]:
# Create a browser instance using splinter
from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\damin\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [7]:
# Visit the Mars News url
MarsNews_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(MarsNews_url)

# Create HTML object
html = browser.html

# Parse HTML with BeautifulSoup
soup = bs(html, 'html.parser')

# Save the news title as variable
news_title = soup.find('div', class_='content_title').text
print(news_title)

# Save the paragraph text as variable
news_p = soup.find('div', class_='article_teaser_body')
print(news_p)


Mars Now
<div class="article_teaser_body">NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.</div>


### JPL Mars Space Images - Featured Image

In [8]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image.
# Create a browser instance using splinter
from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\damin\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [9]:
# Visit the Mars News url
jpl_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
browser.visit(jpl_url)

# Create HTML object
html = browser.html

# Parse HTML with BeautifulSoup
soup = bs(html, 'html.parser')

# Save the hero image url as variable
base_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'

#Find the src of the correct image (hero Image)
relative_image_path = soup.find_all('img')[1]["src"]

# Complete the featured image url by adding the base url ---
featured_image_url = base_url + relative_image_path
featured_image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg'

### Mars Facts

In [10]:
#use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
url = 'https://space-facts.com/mars/'

#Use Pandas to convert the data to a HTML table string.
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [13]:
# Find the correct DataFrame in the list of DataFrames as assign it to `mars_df`
mars_df = tables[0]
mars_df.columns = ['Description', 'Value']
mars_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


### Mars Hemispheres

In [14]:
# Find location of all hemisphere titles and thumbnails
items = soup.find_all('div', class_='item')

#Make empty lists to store the hemisphere name and thumbnail url
names = []
urls = []

#Loop through items and store hemisphere name and thumbnail url in lists
for item in items:
    urls.append(base_url + item.find('a')['href'])
    names.append(item.find('h3').text.strip())
print(names + urls)

[]


In [17]:
# Visit https://astrogeology.usgs.gov
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

#Make empty list to store the hemisphere name and image url as dictionaries
hemisphere_dict_list = []

# Get all elements that contain image information
items = soup.find_all("div", class_="item")

# Iterate through each image
for item in items:
    # Get the Hemisphere name
    name = item.find("h3").text
    # Set up to go to hemisphere pages to get full image url
    end_url = item.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_url
    # Visit Individual hemisphere page   
    browser.visit(image_link)
    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")
    # Get full image url
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    # Save hemisphere name and image url to empty list as 4 dictionaries
    hemisphere_dict_list.append({"Name": name, "img_url": image_url})

# Print image title and url
print(hemisphere_dict_list)

[{'Name': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'Name': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'Name': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'Name': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [18]:
# Create dictionary for all info scraped from sources above
mars_info={
    "news_title":news_title,
    "news_p":news_p,
    "featured_image_url":featured_image_url,
    "fact_table":mars_df,
    "hemisphere_images":hemisphere_dict_list
}

In [19]:
# Close the browser after scraping
browser.quit()

In [20]:
mars_info

{'news_title': 'Mars Now',
 'news_p': <div class="article_teaser_body">NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.</div>,
 'featured_image_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg',
 'fact_table':             Description                          Value
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
 'hemisphere_images': [{'Name': 'Cerberus Hemisphere Enhanced',
   'img_url': 'https://astropedia.a